### FML Project - Dancea Radu Ioan

In [32]:
import pandas as pd

Load the data.

In [33]:
DATA_PATH = "data/"

results = pd.read_csv(DATA_PATH + "results.csv")
races = pd.read_csv(DATA_PATH + "races.csv")
qualifying = pd.read_csv(DATA_PATH + "qualifying.csv")

In [35]:
print(results.shape)
print(results.head())
print(races.shape)

(26759, 18)
   resultId  raceId  driverId  constructorId number  grid position  \
0         1      18         1              1     22     1        1   
1         2      18         2              2      3     5        2   
2         3      18         3              3      7     7        3   
3         4      18         4              4      5    11        4   
4         5      18         5              1     23     3        5   

  positionText  positionOrder  points  laps         time milliseconds  \
0            1              1    10.0    58  1:34:50.616      5690616   
1            2              2     8.0    58       +5.478      5696094   
2            3              3     6.0    58       +8.163      5698779   
3            4              4     5.0    58      +17.181      5707797   
4            5              5     4.0    58      +18.014      5708630   

  fastestLap rank fastestLapTime fastestLapSpeed  statusId  
0         39    2       1:27.452         218.300         1  
1     

Keep only relevant columns

In [36]:
results = results[[
    "raceId",
    "driverId",
    "constructorId",
    "grid",
    "positionOrder"
]]

races = races[[
    "raceId",
    "year",
    "circuitId"
]]

qualifying = qualifying[[
    "raceId",
    "driverId",
    "position"
]].rename(columns={"position": "qualifying_position"})

In [37]:
results["top3"] = (results["positionOrder"] <= 3).astype(int)

In [38]:
df = results.merge(races, on="raceId", how="left")
df = df.merge(
    qualifying,
    on=["raceId", "driverId"],
    how="left"
)

In [39]:
print(df.shape)
df.head()

(26759, 9)


,raceId,driverId,constructorId,grid,positionOrder,top3,year,circuitId,qualifying_position
0,18,1,1,1,1,1,2008,1,1.0
1,18,2,2,5,2,1,2008,1,5.0
2,18,3,3,7,3,1,2008,1,7.0
3,18,4,4,11,4,0,2008,1,12.0
4,18,5,1,3,5,0,2008,1,3.0
